# Just a short notebook that calculate the SFR indicator adjustment based according to different IMF.  The idea of the SFR indicators (such as H$\alpha$, FIR) here is that these singals are emitted from the massive stars which has short lightspan ( < 50Myr), so tracing the massive stars autometically trace the stars formed within 50 Myr.  


## Say the Kennicutt law has: 

## SFR(IR) = 4.5 $\times$ 10$^{-44}$  L$_{TIR}$ ,
## where L$_{TIR}$ is the total infrared luminosity (from 8 $\mu$m to 1000 $\mu$m) in the unit of [erg/s] , under the assumption of Salpeter IMF. 


## Since the O stars (massive stars) are dominating the emission of the SFR indicators, we can distill the calibration factor (4.5$\times$10$^{-44}$) as: 
## SFR(IR) = A$\times (\frac{M_{total}}{L_{TIR}})$  L$_{TIR}$
## = B $\times ( \frac{M_{total}}{M_{Ostar}}) (\frac{M_{Ostar}}{L_{TIR}})$ L$_{TIR}$. 
## where A, B, and the mass to light ratio ($\frac{M_{Ostar}}{M_{L_{TIR}}}$) are constants with respect to different IMF.   (Since the O stars are the dominating source of the indicators' emission)

## The term $ \frac{M_{total}}{M_{Ostar}}$ is where we abjust based on the IMF. 

## $ \frac{M_{total}}{M_{Ostar}} = \frac{\int^{100}_{0.1} m  \xi dm}{\int^{100}_{10} m  \xi dm}$.  here 100 solar mass is the upper limit of the total stellar population, and 10 solar mass is the lower limit of the massive stars (O stars). 


## If we want to convert the Kennicutt law of Salpeter IMF to Chabrier IMF,  

## we should adjust the calibration as : 

## SFR(IR)$_{Chabrier}$ =  4.5 $\times$ 10$^{-44} \times$ $(\frac{M_{total}}{M_{Ostar}})_{Chabrier}$ /  $(\frac{M_{total}}{M_{Ostar}})_{Salpeter}$L$_{TIR}$ 

## $(\frac{M_{total}}{M_{Ostar}})_{Chabrier}$ /  $(\frac{M_{total}}{M_{Ostar}})_{Salpeter}$ is the factor between the IMFs

In [1]:
import numpy as np
from scipy.integrate import quad




def chabrier_imf(m):


    m = np.array(m, ndmin=1)
    imf = np.zeros_like(m)

    # Parameters for Chabrier IMF
    mc = 0.08
    sigma = 0.69
    A = 0.158  
    B = A * np.exp(- (np.log10(1) - np.log10(mc))**2 / (2 * sigma**2)) / 1  # continuity at m=1

    log10m = np.log10(m)

    # IMF for m <= 1 M_sun (log-normal)
    mask_low = m <= 1
    imf[mask_low] = A * np.exp(- (log10m[mask_low] - np.log10(mc))**2 / (2 * sigma**2)) / m[mask_low]

    # IMF for m > 1 M_sun (power-law)
    mask_high = m > 1
    imf[mask_high] = B * m[mask_high]**(-2.3)
    



    return imf


def kroupa_imf(m):
    if m < 0.08:
        return 1.0 * m**(-0.3)
    elif m < 0.5:
        return (0.08**(1.0)) * m**(-1.3)
    else:
        return (0.08**(1.0)) * (0.5**(-1.3 + 2.3)) * m**(-2.3)


def salpeter_imf(m):

    m = np.array(m, ndmin=1)
    alpha = 2.35
    A = 1.0  # Default (unnormalized)

    imf = A * m**(-alpha)


    return imf 


In [2]:

norm_sal = quad(salpeter_imf, 0.1, 100)[0]
int1_sal = quad(lambda m: m * salpeter_imf(m)/norm_sal, 0.1, 100)[0]
int2_sal = quad(lambda m: m * salpeter_imf(m)/norm_sal, 10, 100)[0]


norm_kro = quad(kroupa_imf, 0.1, 100)[0]
int1_kro = quad(lambda m: m * kroupa_imf(m)/norm_kro, 0.1, 100)[0]
int2_kro = quad(lambda m: m * kroupa_imf(m)/norm_kro, 10, 100)[0]


norm_cha = quad(chabrier_imf, 0.1, 100)[0]
int1_cha = quad(lambda m: m *chabrier_imf(m)/norm_cha, 0.1, 100)[0]
int2_cha = quad(lambda m: m *chabrier_imf(m)/norm_cha, 10, 100)[0]


In [3]:
4.5e-44 *  ( int1_cha/int2_cha )/ ( int1_sal/int2_sal ) 

2.745651279496629e-44

## So the Kennicutt law in Chabrier IMF is SFR  = 2.74$\times$10$^{-44}$ L$_{IR}$ 